## Overview

KerasNLP makes it very easy to create simple model pipelines at a very fast rate. In this guide we create a simple text classification pipeline from scratch including augmentation, model building etc.

## Imports & setup

This tutorial requires you to have KerasNLP installed:

```shell
pip install keras-nlp
```

We begin by importing all required packages:

In [1]:
!pip install keras-nlp wandb


In [2]:
import numpy as np
import skimage.io as io
import random
import os
import cv2
import pandas as pd
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from glob import glob
from scipy.io import loadmat
import matplotlib.pyplot as plt
import keras_nlp
import tensorflow as tf
import keras
from keras.layers import *
from keras import Sequential
import matplotlib.pyplot as plt

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


Using TensorFlow backend


## Data loading

This guide uses the
[Quora Insincere Questions Classification Dataset](https://www.kaggle.com/competitions/quora-insincere-questions-classification/data)
for demonstration purposes.

To get started, we first load the dataset:


In [3]:
df = pd.read_csv('/kaggle/input/quora-insincere-questions-classification/train.csv')
df

,qid,question_text,target
0,00002165364db923c7e6,How did Quebec nationalists see their province...,0
1,000032939017120e6e44,"Do you have an adopted dog, how would you enco...",0
2,0000412ca6e4628ce2cf,Why does velocity affect time? Does velocity a...,0
3,000042bf85aa498cd78e,How did Otto von Guericke used the Magdeburg h...,0
4,0000455dfa3e01eae3af,Can I convert montra helicon D to a mountain b...,0
...,...,...,...
1306117,ffffcc4e2331aaf1e41e,What other technical skills do you need as a c...,0
1306118,ffffd431801e5a2f4861,Does MS in ECE have good job prospects in USA ...,0
1306119,ffffd48fb36b63db010c,Is foam insulation toxic?,0
1306120,ffffec519fa37cf60c78,How can one start a research project based on ...,0


In [4]:
text = df['question_text'].tolist()
target = df['target'].tolist()

In [5]:
from kaggle_secrets import UserSecretsClient
import wandb
user_secrets = UserSecretsClient()
secret_value_0 = user_secrets.get_secret("api_key")
wandb.login(key = secret_value_0)

wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [6]:
run = wandb.init(project="quora")
table = wandb.Table(data=df)
run.log({'data':table})
run.finish()

wandb: Currently logged in as: tensorgirl. Use `wandb login --relogin` to force relogin
wandb: wandb version 0.16.4 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade
wandb: Tracking run with wandb version 0.16.1
wandb: Run data is saved locally in /kaggle/working/wandb/run-20240317_201746-hqidhmcb
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run fanciful-capybara-6
wandb: ⭐️ View project at https://wandb.ai/tensorgirl/quora
wandb: 🚀 View run at https://wandb.ai/tensorgirl/quora/runs/hqidhmcb
wandb:                                                                                
wandb: 🚀 View run fanciful-capybara-6 at: https://wandb.ai/tensorgirl/quora/runs/hqidhmcb
wandb: Synced 5 W&B file(s), 1 media file(s), 1 artifact file(s) and 0 other file(s)
wandb: Find logs at: ./wandb/run-20240317_201746-hqidhmcb/logs


## Model Building

We use the pretrained `Roberta Classifier` from the KerasNLP to build a simple text classifier.

In [7]:
from wandb.keras import WandbMetricsLogger
run = wandb.init(project="quora",name = 'model_training')

classifier = keras_nlp.models.RobertaClassifier.from_preset(
    "roberta_base_en",
    num_classes=2,
)
classifier.backbone.trainable = False

history = classifier.fit(x=text[:5000], y=target[:5000], verbose =1, epochs=1,batch_size=16,callbacks=[WandbMetricsLogger()])

wandb: wandb version 0.16.4 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade
wandb: Tracking run with wandb version 0.16.1
wandb: Run data is saved locally in /kaggle/working/wandb/run-20240317_202026-lvg1zh2s
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run model_training
wandb: ⭐️ View project at https://wandb.ai/tensorgirl/quora
wandb: 🚀 View run at https://wandb.ai/tensorgirl/quora/runs/lvg1zh2s
Attaching 'config.json' from model 'keras/roberta/keras/roberta_base_en/1' to your Kaggle notebook...
Attaching 'config.json' from model 'keras/roberta/keras/roberta_base_en/1' to your Kaggle notebook...
Attaching 'model.weights.h5' from model 'keras/roberta/keras/roberta_base_en/1' to your Kaggle notebook...
Attaching 'tokenizer.json' from model 'keras/roberta/keras/roberta_base_en/1' to your Kaggle notebook...
Attaching 'assets/tokenizer/vocabulary.json' from model 'keras/roberta/keras/roberta_base_en/1' to your Kaggle notebook...
Attaching 'a

313/313 [==============================] - 388s 1s/step - loss: 0.1720 - sparse_categorical_accuracy: 0.9382


In [8]:
run.finish()

wandb:                                                                                
wandb: 
wandb: Run history:
wandb:                       epoch/epoch ▁
wandb:               epoch/learning_rate ▁
wandb:                        epoch/loss ▁
wandb: epoch/sparse_categorical_accuracy ▁
wandb: 
wandb: Run summary:
wandb:                       epoch/epoch 0
wandb:               epoch/learning_rate 2e-05
wandb:                        epoch/loss 0.17204
wandb: epoch/sparse_categorical_accuracy 0.9382
wandb: 
wandb: 🚀 View run model_training at: https://wandb.ai/tensorgirl/quora/runs/lvg1zh2s
wandb: Synced 6 W&B file(s), 0 media file(s), 0 artifact file(s) and 0 other file(s)
wandb: Find logs at: ./wandb/run-20240317_202026-lvg1zh2s/logs


In [9]:
classifier.predict([text[0]])

1/1 [==============================] - 8s 8s/step


array([[-0.69298095,  0.8235075 ]], dtype=float32)